# Network

Goal of this part: Train an Autoencoder on our scattering patterns.

Steps:
1. Define the architecture of the *encoder* and *decoder*
2. Select *hyper parameters* (learning rate, optimizer, ..) for the training
3. Load the data (with the data loader that we used before)
4. Train our model
5. Look at the results:

Outlook:
- Fitting: What happens during the training?
- Dimensionality reduction: Using the latent space for data exploration

In [1]:
# Optional, if running on Google Colab, install the relevant libraries
#!python -m pip install torch torchvision

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

/Users/wiebke/Documents/Projects/2023.08 ML User Meeting/als_ml_tutorial/tutorial-env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Basic Neural Network Setup in PyTorch

[`nn.Module`](https://pytorch.org/docs/stable/generated/torch.nn.Module.html) is the base class for neural network models. 
When defining your own model, the class you create for the model should be a subclass of this class. It should initialize its super class and overwrite the `forward` function. 

```python
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        # Definition of your model
        # For simple models you can use pre-defined layers 
        # available in PyTorch and wrap them in nn.Sequential()
        # Typical in an Autoencoder: 
        # - Fully connected layers: 
        #   nn.Linear(input_dim, output_dim)
        # - Activation functions: 
        #   nn.ReLU(), nn.Sigmoid(), nn.Tanh(), nn.LeakyReLU().  
        self.model = nn.Sequential(nn.Linear(64*64, 32), nn.nn.ReLU())

    def forward(self, x):
        # Definition of the forward step
        return self.model(x)

```

# Autoencoder

An autoencoder consists of two parts: the *encoder* and the *decoder*.

In [4]:
class Encoder(nn.Module):
    def __init__(self, input_size=64 * 64, latent_dim=32):
        super(Encoder, self).__init__()

        self.input_size = input_size
        self.latent_dim = latent_dim

        self.encoder = nn.Sequential(nn.Linear(input_size, latent_dim), nn.ReLU())

    def forward(self, x):
        # Feed x (the input image) into the encoder
        return self.encoder(x)

With this configuration the encoder has a number of *weights* or *parameters* that we will later optimize. These can be inspected with the `.parameters()` function.

Changing the size of the latent space will adapt the total number of weights.

In [5]:
encoder = Encoder()
# encoder = Encoder(latent_dim = 40)
for param in encoder.parameters():
    print(type(param), param.size())

<class 'torch.nn.parameter.Parameter'> torch.Size([32, 4096])
<class 'torch.nn.parameter.Parameter'> torch.Size([32])


In [6]:
class Decoder(nn.Module):
    def __init__(self, latent_dim=32, output_size=64 * 64):
        super(Decoder, self).__init__()

        self.latent_dim = latent_dim
        self.output_size = output_size

        self.encoder = nn.Sequential(
            nn.Linear(self.latent_dim, self.output_size), nn.ReLU()
        )

    def forward(self, x):
        # Feed x (the result from the encoder) into the decoder
        return self.decoder(x)

In [7]:
class AutoEncoder(nn.Module):
    def __init__(self, input_size=64 * 64, latent_dim=32):
        super(AutoEncoder, self).__init__()

        self.input_size = input_size
        self.latent_dim = latent_dim

        self.encoder = Encoder(input_size=self.input_size, latent_dim=self.latent_dim)
        self.decoder = Decoder(latent_dim=self.latent_dim, output_size=self.input_size)

    def forward(self, x):
        # Feed x (the input image) through the entire autoencoder
        # i.e. calculate the reconstructed image
        latent_representation = self.encoder(x)
        return self.decoder(latent_representation)

# Hyperparameters

Parameters that affect model architecture and training procedure.

In [8]:
hyper_parameters = {
    "num_epochs": 100,
    "batch_size": 40,
    "learning_rate": 1e-3,
    "latent_dim": 32,
}

# Optimization

We also need to define the *loss* and *optimizer*.

In [9]:
# Initialize the model we just defined
autoencoder = AutoEncoder(input_size=64 * 64, latent_dim=hyper_parameters['latent_dim'])

# Validation using MSE Loss function
loss_function = torch.nn.MSELoss()

# Using an Adam Optimizer with lr = 0.1
optimizer = torch.optim.Adam(
    autoencoder.parameters(), lr=hyper_parameters['learning_rate']
)

AttributeError: 'dict' object has no attribute 'latent_dim'

In [ ]:
dataset = ...
dataloader = DataLoader(dataset, batch_size=hyper_parameters['batch_size'], shuffle=True)

In [ ]:
for epoch in range(hyper_parameters['num_epochs']):
    for data in dataloader:
        input_pattern, _ = data
        # Forward step
        reconstruction = autoencoder(input_pattern)
        loss = loss_function(reconstruction, input_pattern)
        # Backward step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()